In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt

In [3]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [5]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [6]:
transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [7]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

In [9]:
cust_data_2020_9 = cust_data.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index()

In [10]:
cust_data_2020_9.head()

,customer_id,prods
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[176209023, 568601006, 568601043, 607642008, 6..."
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[351484002, 541518023, 578020002, 663713001, 7..."
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[448509014, 715624008, 719530003, 726925001, 7..."
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,"[181448103, 309864002, 309864013, 380609037, 4..."
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[297791001, 304692003, 536933002, 668537001, 6..."


In [ ]:
customer_features
1. Age
2. 

In [15]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [14]:
article_df.nunique()

article_id                      105542
product_code                     47224
prod_name                        45875
product_type_no                    132
product_type_name                  131
product_group_name                  19
graphical_appearance_no             30
graphical_appearance_name           30
colour_group_code                   50
colour_group_name                   50
perceived_colour_value_id            8
perceived_colour_value_name          8
perceived_colour_master_id          20
perceived_colour_master_name        20
department_no                      299
department_name                    250
index_code                          10
index_name                          10
index_group_no                       5
index_group_name                     5
section_no                          57
section_name                        56
garment_group_no                    21
garment_group_name                  21
detail_desc                      43404
dtype: int64

In [16]:
article_df['product_group_name'].drop_duplicates()

0           Garment Upper body
3                    Underwear
6               Socks & Tights
8           Garment Lower body
28                 Accessories
36                       Items
37                   Nightwear
64                     Unknown
107        Underwear/nightwear
124                      Shoes
134                   Swimwear
204          Garment Full body
6426                  Cosmetic
26032         Interior textile
31745                     Bags
62949                Furniture
84758    Garment and Shoe care
95767                      Fun
97586               Stationery
Name: product_group_name, dtype: object

In [ ]:
top_12_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(12)

top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)